In [ ]:
# https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html

# PPO
## Double Q Learning (This is technically Double Q learning with a soft update)
# TD Learning (TD-Gammon)
# World Models


import os
import sys
from collections import namedtuple, deque, defaultdict
from typing import List, Tuple
from itertools import count

import random
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import gymnasium as gym

print("GPU Available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [ ]:
env = gym.make("CartPole-v1")
env = gym.make("MountainCarContinuous-v0") # Box not supported
env = gym.make("Acrobot-v1") # Sparse rewards.
env = gym.make("MountainCar-v0") # Sparse rewards 

In [ ]:
# Replay Memory 
Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))


class ReplayMemory:
    def __init__(self, memory_capacity: int):
        self.mem_cap = memory_capacity
        self.reset_memory()
        
    def __len__(self):
        return len(self.memory)
    
    def reset_memory(self):
        self.memory = deque([], maxlen = self.mem_cap)
    
    def append(self, *args) -> None: 
        self.memory.append(Transition(*args))
        
    def sample(self, batchsize: int) -> List[Transition]:
        return random.sample(self.memory, batchsize)
    
    def clear(self):
        self.reset_memory()

In [ ]:
class DQN(nn.Module):
    def __init__(self, n_obs, n_actions): 
        super(type(self), self).__init__()
        self.layer_size = 256
        self.n_hidden_layers = 2
        if self.n_hidden_layers < 0: 
            raise ValueError("Network must have at least 0 layers (input and output)")
#         self.layers = []
#         self.layers.append(nn.Linear(n_obs, self.layer_size))
#         for _ in range(1, self.n_layers - 1):
#             self.layers.append(nn.Linear(self.layer_size, self.layer_size))
#         self.layers.append(nn.Linear(self.layer_size, n_actions))
        
        self.input_layer = nn.Linear(n_obs, self.layer_size)
        self.hidden1 = nn.Linear(self.layer_size, self.layer_size) 
        self.hidden2 = nn.Linear(self.layer_size, self.layer_size)
        self.output_layer = nn.Linear(self.layer_size, n_actions)
         
        self.layers = [self.input_layer, self.hidden1, self.hidden2, self.output_layer]
    
    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
        return x

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
MIN_EARLY_STOP_EP = 300
EARLY_STOP_MEANLEN = 50
EARLY_STOP_DECR_PERIODS = 20

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0
def select_action(net, state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        np.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


def plot_means(values_t: torch.tensor, meanlen: int):
    # Take 100 episode averages and plot them too
    if len(values_t) >= meanlen:
        means = values_t.unfold(0, meanlen, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(meanlen - 1), means))
        plt.plot(means.numpy())
    
def plot_values(values, show_result=False, stop_episode=None):
    plt.figure(1)
    values_t = torch.tensor(values, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(values_t.numpy())
    plot_means(values_t, 100)
    plot_means(values_t, EARLY_STOP_MEANLEN)
    
    if stop_episode is not None: 
        plt.axvline(stop_episode, c = 'r')

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())
            
def stop_early(values: List[int], n_periods: int, meanlen: int) -> bool:
    # Stop early if mean duration has decreased for the last 10 durations
    # Take 100 episode averages and plot them too
    values_t = torch.tensor(values, device=device, dtype=torch.float)
    if len(values_t) > n_periods:
        means = values_t.unfold(0, meanlen, 1).mean(1).view(-1)
        prev_t = means[-n_periods-1: -1]
        cur_t = means[-n_periods:]
        decreasing_t = torch.all(prev_t >= cur_t)
#         print(cur_t, decreasing_t)
        return decreasing_t.item()
        

In [ ]:
# Training
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

    
    
    
    
if torch.cuda.is_available():
    num_episodes = 1250
else:
    num_episodes = 50

early_stop = False
stop_episode = None
episode_durations = []
episode_rewards = []
for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    total_reward = 0
    for t in count():
        action = select_action(policy_net, state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        total_reward += reward
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.append(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        

        if done:
            episode_durations.append(t + 1)
            episode_rewards.append(total_reward)
#             plot_values(episode_durations)
            plot_values(episode_rewards)
    
            #Early stopping 
            if i_episode >= MIN_EARLY_STOP_EP: 
                early_stop = stop_early(episode_durations, n_periods = EARLY_STOP_DECR_PERIODS,\
                                        meanlen = EARLY_STOP_MEANLEN)
            break
    if early_stop:
        stop_episode = i_episode 
        episode_rewards.append(total_reward)
        print(f"Stopping at {stop_episode}")
        break
        
if stop_episode is None: 
    stop_episode = num_episodes - 1
print('Complete')
# plot_values(episode_durations, show_result=True, stop_episode = stop_episode)
plot_values(episode_rewards, show_result = True, stop_episode = stop_episode)

plt.show()

In [ ]:
if torch.cuda.is_available():
    num_episodes = 100
else:
    num_episodes = 20

for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(target_net, state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
            
        # Move to the next state
        state = next_state

        if done:
            episode_rewards.append(total_reward)
            episode_durations.append(t + 1)
#             plot_values(episode_durations, stop_episode = stop_episode)
            plot_values(episode_rewards, stop_episode = stop_episode)
            break

print('Complete')
# plot_durations(episode_durations, show_result=True, stop_episode = stop_episode)
plot_values(episode_rewards, show_result = True, stop_episode = stop_episode)

plt.show()

In [ ]:
plt.ioff()